## 注意力机制Attention:

### 简介:

这一部分是Transformer模型的核心部分,以下部分逐步给出实现过程中可能用到的一些矩阵运算的原理， 以下代码均不需要大家实现,希望大家阅读代码以及下列文档中的信息:

https://arxiv.org/abs/1706.03762

https://jalammar.github.io/illustrated-transformer/

理解Attention的运行机制以及实现过程的数学技巧，完成最后的主文件中的HeadAttention(),MultiHeadAttention()部分。

我们虚构一组输入数据的Embedding用于这部分讲解：

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
B, T, C = 1, 8, 16   ## B: batch size 一次训练的数据量, T: context length 前文token数, C: embedding length 隐变量长度
inputData = torch.rand(size=(B,T,C))

for i in range(T):
    print(f"Embedding of {i}th position:\n {inputData[0,i]}")


Embedding of 0th position:
 tensor([0.6773, 0.7098, 0.4272, 0.5622, 0.2731, 0.7304, 0.3566, 0.7356, 0.8300,
        0.5739, 0.4327, 0.9170, 0.7448, 0.3570, 0.6208, 0.6920])
Embedding of 1th position:
 tensor([0.2382, 0.6290, 0.4834, 0.7422, 0.4020, 0.8890, 0.5385, 0.0952, 0.0054,
        0.4483, 0.9280, 0.5547, 0.8022, 0.1790, 0.1831, 0.2948])
Embedding of 2th position:
 tensor([0.7674, 0.9722, 0.1052, 0.7407, 0.5425, 0.7726, 0.2650, 0.8362, 0.7750,
        0.8019, 0.6017, 0.3437, 0.3333, 0.0733, 0.3888, 0.0779])
Embedding of 3th position:
 tensor([0.9703, 0.4540, 0.9663, 0.0288, 0.4122, 0.0723, 0.8923, 0.4230, 0.2880,
        0.2617, 0.0920, 0.4588, 0.9402, 0.7001, 0.0385, 0.4353])
Embedding of 4th position:
 tensor([0.0782, 0.9565, 0.7143, 0.6742, 0.6731, 0.1151, 0.4827, 0.6577, 0.2909,
        0.0740, 0.2749, 0.8979, 0.9204, 0.7161, 0.1784, 0.6771])
Embedding of 5th position:
 tensor([0.9091, 0.2057, 0.3678, 0.1444, 0.1281, 0.7960, 0.4960, 0.5763, 0.6024,
        0.3228, 0.9494, 0.5

Attention从直观上可以理解为对前文各个位置信息的融合以获得当前语境所需的信息。 一个最简单的融合方式为对前文Embedding加权求和作为当前位置的信息。

我们计算第i个位置的融合后的embedding:

假设前i个位置的embedding的权重相同，均为1/i，即更新后第i个位置embedding为前文所有位置embedding的平均值：

In [2]:
def Attention_version1(contextEmbeddings):
    for i in range(T):
        context_embeddings = contextEmbeddings[0,:i+1,:] ## shape [i+1, C]
        new_embedding_for_i = torch.mean(context_embeddings,dim=0)
        contextEmbeddings[0,i] = new_embedding_for_i
    return contextEmbeddings

print("Embedding of Data after aggregate context embedding:\n", Attention_version1(inputData))

Embedding of Data after aggregate context embedding:
 tensor([[[0.6773, 0.7098, 0.4272, 0.5622, 0.2731, 0.7304, 0.3566, 0.7356,
          0.8300, 0.5739, 0.4327, 0.9170, 0.7448, 0.3570, 0.6208, 0.6920],
         [0.4578, 0.6694, 0.4553, 0.6522, 0.3375, 0.8097, 0.4475, 0.4154,
          0.4177, 0.5111, 0.6804, 0.7358, 0.7735, 0.2680, 0.4019, 0.4934],
         [0.6342, 0.7838, 0.3292, 0.6517, 0.3844, 0.7709, 0.3563, 0.6624,
          0.6742, 0.6290, 0.5716, 0.6655, 0.6172, 0.2328, 0.4705, 0.4211],
         [0.6849, 0.6543, 0.5445, 0.4737, 0.3518, 0.5958, 0.5132, 0.5591,
          0.5525, 0.4939, 0.4442, 0.6943, 0.7689, 0.3895, 0.3829, 0.5104],
         [0.5065, 0.7548, 0.4941, 0.6028, 0.4040, 0.6044, 0.4313, 0.6060,
          0.5531, 0.4564, 0.4808, 0.7821, 0.7649, 0.3927, 0.4109, 0.5588],
         [0.6450, 0.6296, 0.4364, 0.5145, 0.3132, 0.7179, 0.4335, 0.5925,
          0.6050, 0.4978, 0.5932, 0.7204, 0.7233, 0.3545, 0.4652, 0.5597],
         [0.5458, 0.6569, 0.4661, 0.6191, 0.3524, 0.

我们将上述的mean操作换为等价的矩阵运算，以i=3 为例：

new_embedding_for_3 = torch.mean(contextEmbeddings[0,:3+1],dim=0)

等价于(@ 是矩阵乘法):

new_embedding_for_3 = contextEmbeddings[0] @ torch.tensor([1/4,1/4,1/4,1/4,0,0,0,0])

In [3]:
def Attention_version2(contextEmbeddings):
    for i in range(T):
        weight = torch.cat((torch.ones(i+1) / (i+1),torch.zeros(T-i-1,dtype=torch.float)),dim=0)
        contextEmbeddings[0,i] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version2: ",torch.all(Attention_version1(inputData) == Attention_version2(inputData)).item())

Attention_version1 equivalent to Attention_version2:  True


接下来我们用矩阵运算进一步简化上述运算，移除其中的for循环:

其中 weight = torch.tril(torch.ones(T,T)) 得到:

[[1., 0., 0., 0., 0., 0., 0., 0.],

 [1., 1., 0., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 1.]]
 
表示前文的求和权重相同都为一。

weight = weight.masked_fill(weight==0,float("-inf"))

weight = F.softmax(weight)

这两行用于归一化weight,即每一次加权求和的权重和为1，具体详见Softmax公式,我们可得到：

[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],

[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],

[0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],

[0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]


In [ ]:
def Attention_version3(contextEmbeddings):
    B, T, C = contextEmbeddings.shape
    weight = torch.tril(torch.ones(T,T))
    print("weight of context embeddings:\n",weight)
    weight = weight.masked_fill(weight==0,float("-inf"))
    weight = F.softmax(weight,dim=1)
    print("weight of context embeddings after regularization:\n",weight)
    contextEmbeddings[0] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version3: ",torch.all(Attention_version1(inputData) == Attention_version3(inputData)).item())

最后，我们确定计算weight的方法，上述三个版本的weight都是假定所有前文信息的重要程度相同,在大语言模型中，我们希望有一个灵活的方式计算前文信息对应当前语境的重要程度，为此Transformer引入了Query，Key，Value:

其中Query可以理解为当前语境对于前文信息的需求，Key可以理解为前文包含信息的索引，Value为前文所包含的信息。

Query 和 Key 用来计算信息融合的weight.

如何计算Query和Key，并用他们计算weight对Value加权求和是这次实验的重点内容，这里不能给出大家具体代码，希望大家参见Attention is All you need原论文以及助教提供的文档最后的参考链接学习这部分。

利于Query和Key得出的是信息相关性，我们需要遮盖住下文的信息(生成第i个token时，只可以使用0到i-1处的信息)，并且要对相关性归一化使之可以作为weight。这里利于Attension_version3()中的结论给出如何对计算出来的相关性加掩码和归一化:


In [ ]:
def weight_mask_and_normalization(weight):
    tril = torch.tril(torch.ones_like(weight))
    weight = weight.masked_fill(tril == 0, float("-inf"))
    weight = F.softmax(weight,dim=-1)
    return weight

weight = torch.rand(T,T)
print("weight before mask and normalize:\n",weight)
print("weight after mask and normalize:\n",weight_mask_and_normalization(weight))